<a href="https://colab.research.google.com/github/SarvaniAnkala/Text_Summarisation/blob/main/NLP_TEXT_SUMMARISATION2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets nltk torch pandas sklearn spacy sumy
import pandas as pd
import numpy as np
import torch
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer, PegasusForConditionalGeneration, PegasusTokenizer


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
!pip install scikit-learn
import sklearn


In [3]:
# Load CompScholar Dataset
comp_df = pd.read_csv('CompScholar.csv')
print(comp_df.head())  # Display first few rows


   Paper Id                                        Paper Title  \
0         1  Multi-document Summarization via Deep Learning...   
1         2  NLP based Machine Learning Approaches for Text...   
2         3  Abstractive text summarization using LSTM-CNN ...   
3         4  DEXPERTS: Decoding-Time Controlled Text Genera...   
4         5     A Survey of Knowledge-enhanced Text Generation   

                                           Key Words  \
0  Multi-document summarization (MDS), Deep learn...   
1  Text summarization, Abstractive and extractive...   
2  Text mining . Abstractive text summarization ....   
3  Natural language generation, Controlled text g...   
4  text-to-text generation, natural language proc...   

                                            Abstract  \
0  Multi-document summarization (MDS) is an effec...   
1  Due to the plethora of data available today, t...   
2   Abstractive Text Summarization (ATS), which i...   
3  Despite recent advances in natural lang

In [4]:
!pip install datasets


  Using cached datasets-3.4.1-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machi

In [5]:
from huggingface_hub import notebook_login
notebook_login()


In [6]:
from datasets import load_dataset
# Step 1: Load the Training Datasets (ArXiv & PubMed)
train_dataset_arxiv = load_dataset("ccdv/arxiv-summarization", split="train[:10%]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [7]:
# Load the PubMed summarization dataset
train_dataset_pubmed = load_dataset("ccdv/pubmed-summarization", split="train[:10%]")

README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/59.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/58.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [8]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("CompScholar.csv")  # Ensure the file is in the correct directory
eval_dataset = Dataset.from_pandas(df)


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
# Step 3: Choose the Model (T5, BART, or PEGASUS)
MODEL_NAME = "facebook/bart-large-cnn"  # You can also use "t5-small" or "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [10]:
# Step 4: Preprocessing Function
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(examples["abstract"], max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
# Step 5: Tokenization
train_dataset_arxiv = train_dataset_arxiv.map(preprocess_function, batched=True)
train_dataset_pubmed = train_dataset_pubmed.map(preprocess_function, batched=True)

Map:   0%|          | 0/20304 [00:00<?, ? examples/s]

Map:   0%|          | 0/11992 [00:00<?, ? examples/s]

In [12]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["Document"]]  # Use "Document" for articles
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    labels = tokenizer(examples["Summary"], max_length=256, truncation=True, padding="max_length")  # Use "Summary" for summaries
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [13]:
print(eval_dataset.column_names)


['Paper Id', 'Paper Title', 'Key Words', 'Abstract', 'Conclusion', 'Document', 'Paper Type', 'Summary', 'Topic', 'OCR', 'labels']


In [14]:
eval_dataset = eval_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/371 [00:00<?, ? examples/s]

In [15]:
from datasets import concatenate_datasets

# Step 6: Merge Training Datasets
train_dataset = concatenate_datasets([train_dataset_arxiv, train_dataset_pubmed])


In [16]:
train_dataset = train_dataset.select(range(int(len(train_dataset) * 0.1)))

In [17]:
print(train_dataset)

Dataset({
    features: ['article', 'abstract', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3229
})


In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # Evaluate every few steps instead of every epoch
    save_strategy="steps",  # Save checkpoints every few steps to prevent loss of progress
    save_steps=1000,  # Save model every 1000 steps instead of every epoch
    per_device_train_batch_size=2,  # Reduce batch size to 2
    per_device_eval_batch_size=2,  # Reduce batch size to 2
    num_train_epochs=2,  # Reduce epochs (fewer passes over dataset)
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=250,  # Log more frequently for debugging
    gradient_accumulation_steps=8,  # Accumulate gradients for effective larger batch size
    fp16=True,  # Enable mixed precision (faster & less memory usage)
    push_to_hub=False,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# Step 8: Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

<ipython-input-19-70f684338373>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ankaladurgasarvani (ankaladurgasarvani-iiest-shibpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
250,2.173400,0.808631


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=402, training_loss=2.040210780812733, metrics={'train_runtime': 1134.7984, 'train_samples_per_second': 5.691, 'train_steps_per_second': 0.354, 'total_flos': 1.3960487846805504e+16, 'train_loss': 2.040210780812733, 'epoch': 1.9956656346749226})

In [23]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


In [25]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c14827544647e00ab5fc370c92494c3a708aa611490531e450642d687cbe4693
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [26]:
from evaluate import load
import numpy as np

rouge = load("rouge")


In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {key: value.mid.fmeasure * 100 for key, value in result.items()}

In [30]:
# Step 10: Evaluate the Model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

Evaluation Results: {'eval_loss': 0.7905768752098083, 'eval_runtime': 19.7808, 'eval_samples_per_second': 18.756, 'eval_steps_per_second': 9.403, 'epoch': 1.9956656346749226}


In [33]:
import torch
import os

# Check GPU type
gpu_info = !nvidia-smi
gpu_info = "\n".join(gpu_info)
print("🖥️ GPU Info:\n", gpu_info)

# Check total memory (RAM)
ram_info = !free -h
ram_info = "\n".join(ram_info)
print("\n🛑 RAM Info:\n", ram_info)

# Check available CUDA memory
if torch.cuda.is_available():
    print(f"\n🔥 CUDA Memory Available: {torch.cuda.mem_get_info()[0] / 1e9:.2f} GB")
else:
    print("\n⚠️ No GPU found, using CPU.")


🖥️ GPU Info:
 Sat Mar 22 19:21:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             35W /   70W |   12100MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------------------